# Import Libraries

In [6]:
import pandas as pd
import psycopg2
import os
from psycopg2.extras import execute_values
from dotenv import load_dotenv


# Load Cleaned Reviews

#

In [7]:
data = pd.read_csv('../data/processed/reviews_final.csv')
data.head()

,Unnamed: 0,review_text,rating,review_date,bank_name,sentiment_score,sentiment_label,themes
0,0,The app makes our life easier. Thank you CBE!,5,2025-11-27,Commercial Bank of Ethiopia,0.000000,Neutral,User Interface & Experience
1,1,this app very bad,1,2025-11-27,Commercial Bank of Ethiopia,-0.910000,Negative,User Interface & Experience
2,2,the most advanced app. but how to stay safe?,5,2025-11-27,Commercial Bank of Ethiopia,0.466667,Neutral,Other
3,3,Good application,4,2025-11-27,Commercial Bank of Ethiopia,0.700000,Positive,User Interface & Experience
4,4,It is nice app,5,2025-11-26,Commercial Bank of Ethiopia,0.600000,Positive,User Interface & Experience


# Connect to PostgreSQL

In [8]:
conn = psycopg2.connect(
    dbname=os.getenv("DB_NAME"),
    user= os.getenv("DB_USER"),
    password=os.getenv("DB_PASS"),
    host=os.getenv("DB_HOST"),
    port=os.getenv("DB_PORT")
)
cur = conn.cursor()
print("Connected to PostgreSQL successfully!")

Connected to PostgreSQL successfully!


# Insert Unique Banks

In [9]:
banks = data[['bank_name']].drop_duplicates()
for _, row in banks.iterrows():
    cur.execute("""
        INSERT INTO banks (bank_name, app_name)
        VALUES (%s, %s)
        ON CONFLICT (bank_name) DO NOTHING;
    """, (row['bank_name'], 'Mobile Banking App'))
conn.commit()
print("Banks inserted successfully!")


Banks inserted successfully!


# Map bank_name -> bank_id

In [10]:
cur.execute('SELECT bank_id, bank_name FROM banks;')
bank_mapping = {name: bank_id for bank_id, name in cur.fetchall()}
print(f"{len(bank_mapping)} banks inserted/mapped successfully.")

3 banks inserted/mapped successfully.


# Prepare Review Rows

In [11]:
review_rows = [
    (
        bank_mapping[row['bank_name']],
        row['review_text'],
        row['rating'],
        row['review_date'],
        row['sentiment_label'],
        row['sentiment_score'],
        'Google Play Store' 
    )
    for _, row in data.iterrows()
]
print(f"Prepared {len(review_rows)} reviews for bulk insert.")


Prepared 1455 reviews for bulk insert.


# Bulk Insert Reviews

In [12]:
insert_query = """
INSERT INTO reviews
(bank_id, review_text, rating, review_date, sentiment_label, sentiment_score, source)
VALUES %s
ON CONFLICT (review_text, review_date, bank_id) DO NOTHING;
"""
execute_values(cur, insert_query, review_rows)
conn.commit()
print("All reviews inserted successfully!")


All reviews inserted successfully!


# Close Connection

In [13]:
cur.close()
conn.close()
print("PostgreSQL connection closed.")


PostgreSQL connection closed.


# Verify Inserted Data 

In [14]:
conn = psycopg2.connect(
    dbname=os.getenv("DB_NAME"),
    user= os.getenv("DB_USER"),
    password=os.getenv("DB_PASS"),
    host=os.getenv("DB_HOST"),
    port=os.getenv("DB_PORT")
)
cur = conn.cursor()

# Count reviews per bank and total reviews in the entire database

In [17]:
cur.execute("""
SELECT b.bank_name, COUNT(r.review_id) AS total_reviews
FROM banks b
LEFT JOIN reviews r ON b.bank_id = r.bank_id
GROUP BY b.bank_name;
""")
for row in cur.fetchall():
    print(row)
cur.execute("SELECT COUNT(*) FROM reviews;")
total = cur.fetchone()[0]
print("Total reviews in database:", total)


('Dashen Bank', 495)
('Bank of Abyssinia', 480)
('Commercial Bank of Ethiopia', 476)
Total reviews in database: 1451


In [23]:
conn.rollback()
print("Transaction reset!")


Transaction reset!


In [24]:
cur.execute("""
UPDATE reviews
SET review_text = 'No review text provided'
WHERE review_text = 'NaN';
""")

In [25]:
query = """
SELECT review_id, bank_id, review_text, review_date
FROM reviews
WHERE review_text IS NULL
   OR review_text = ''
   OR review_text = 'NaN'
   OR review_text = 'NAN';
"""

cur.execute(query)
rows = cur.fetchall()
df_invalid = pd.DataFrame(rows, columns=["review_id", "bank_id", "review_text", "review_date"])
df_invalid



,review_id,bank_id,review_text,review_date


In [26]:
cur.close()
conn.close()
print("PostgreSQL connection closed.")

PostgreSQL connection closed.


# Average rating per bank

In [37]:

conn = psycopg2.connect(
    dbname=os.getenv("DB_NAME"),
    user= os.getenv("DB_USER"),
    password=os.getenv("DB_PASS"),
    host=os.getenv("DB_HOST"),
    port=os.getenv("DB_PORT")
)

cur = conn.cursor()
print("Reconnected successfully!")

Reconnected successfully!


In [30]:
query=""" 
SELECT b.bank_name, ROUND(AVG(r.rating), 2) AS avg_rating
FROM banks b
JOIN reviews r ON b.bank_id = r.bank_id
GROUP BY b.bank_name
ORDER BY avg_rating DESC;

"""
cur.execute(query)
rows = cur.fetchall()
rows

[('Dashen Bank', Decimal('4.08')),
 ('Commercial Bank of Ethiopia', Decimal('3.97')),
 ('Bank of Abyssinia', Decimal('2.78'))]

# Count reviews per sentiment (positive / negative / neutral)

In [31]:
query = """
SELECT sentiment_label, COUNT(*) AS total
FROM reviews
GROUP BY sentiment_label
ORDER BY total DESC;

"""
cur.execute(query)
sentiment_results = cur.fetchall()
sentiment_results

[('Neutral', 1058), ('Positive', 339), ('Negative', 54)]

# Review count by date (detect spikes or missing dates)

In [34]:
query = """
SELECT review_date, COUNT(*) AS reviews_on_day
FROM reviews
GROUP BY review_date
ORDER BY reviews_on_day DESC;
"""
cur.execute(query)
review_datas = cur.fetchall()
review_datas[:10]

[(datetime.date(2025, 4, 21), 103),
 (datetime.date(2025, 10, 28), 25),
 (datetime.date(2025, 4, 22), 21),
 (datetime.date(2025, 8, 12), 18),
 (datetime.date(2025, 8, 22), 17),
 (datetime.date(2025, 11, 5), 16),
 (datetime.date(2025, 11, 10), 15),
 (datetime.date(2025, 8, 20), 14),
 (datetime.date(2024, 11, 12), 14),
 (datetime.date(2025, 9, 27), 13)]

# Detect duplicated reviews

In [38]:
cur.execute("ROLLBACK;")


In [39]:
query = """
SELECT review_text, bank_id, review_date, COUNT(*)
FROM reviews
GROUP BY review_text, bank_id, review_date
HAVING COUNT(*) > 1;
"""
cur.execute(query)
dup_data = cur.fetchall()
df_invalid = pd.DataFrame(dup_data, columns=["bank_id", "review_text", "review_date"])
df_invalid

,bank_id,review_text,review_date
